In [ ]:
# SETUP PARAMETERS
# db_addr='128.199.166.149'
db_addr='172.16.101.172'
# db_addr='localhost'
db_port=27340
db_user_email = 'demo@example.com'
filename_ics = "DEMO_TASK_HISTORY_1114.ics"

In [ ]:
# Access DB. Find user ID for the google ics file.
from pymongo import MongoClient

client = MongoClient(db_addr, db_port)
# print client.database_names()
db = client.test
# print db.collection_names()

myaccount = db.accounts.find_one({'email': db_user_email})
userID = myaccount['_id']
print userID

In [ ]:
import tokenizer
import json

def tzChange(t, tz_from='UTC', tz_to='Asia/Seoul'):
    from datetime import datetime
    from dateutil import tz
    
    from_zone = tz.gettz(tz_from)
    to_zone = tz.gettz(tz_to)
    # Tell the datetime object 't'.
    # datetime objects are 'naive' by default
    t = t.replace(tzinfo=from_zone)

    # Convert time zone
    local = t.astimezone(to_zone)
    return local

def getTimeIndex336(t):
    '''
    t is python datetime object
    '''
    
    # Sunday:0, Saturday:6
    return (t.weekday()+1)%7 * 48 + t.hour * 2 + t.minute/30

def getTimeIndex48(t):
    '''
    t is python datetime object
    '''
    # Sunday:0, Saturday:6
    return t.hour * 2 + t.minute/30


CONST_GOOGLECAL_TOKEN_TYPE = 9999

def createPredTokenFromGCal(userID, text, daytime=-1, weekday=-1):
    import datetime
    objdict = {};
    objdict['loc'] = {'type': 'Point', 'coordinates': [127.04512999999999, 37.5040797]} # u'loc': {u'type': u'Point', u'coordinates': [127.04512999999999, 37.5040797]}
    objdict['text'] = text
    objdict['description'] = ' '
    objdict['userId'] = userID
    
    # Only one filed has meaningful value and the others have value -1.
    objdict['daytime'] = daytime # getTimeIndex48(time)
    objdict['weekday'] = weekday # (time.weekday()+1)%7
    
    objdict['prevType'] = 0 # Dummy
    objdict['__v'] = 0 # Dummy
    objdict['taskId'] = None # ObjectId('562a0c5441121d2d907405ec')
    objdict['duration'] = datetime.datetime(1970, 1, 2, 0, 0) # Dummy
    objdict['type'] = CONST_GOOGLECAL_TOKEN_TYPE
    
    return objdict


def createTaskLogFromGCal(user_id, log_type, time):
    '''
    user_id: ObjectId
    log_type: 200 (start), 500 (complete)
    '''
    import datetime
    objdict = {};

    objdict['userId'] = user_id
    # objdict['taskId'] = ''
    objdict['type'] = log_type;
    objdict['loc'] = {'type': 'Point', 'coordinates': [127.04512999999999, 37.5040797]} # u'loc': {u'type': u'Point', u'coordinates': [127.04512999999999, 37.5040797]}

    return objdict


import vobject
data = open(filename_ics).read()
cal = vobject.readOne(data)

predtokens = []
tasklogs = []
print 'Number of tasks: ', len(cal.vevent_list)
for event in cal.vevent_list:
    print 'Summary: ', event.summary.value
    print 'Description: ', event.description.value
    tokens = tokenizer.permute(tokenizer.extractor(event.summary.value + ' ' + event.description.value))
    print 'Tokens: ', json.dumps(tokens, ensure_ascii=False)

    # Get Time
    stTime_local = tzChange(event.dtstart.value)
    enTime_local = tzChange(event.dtend.value)
    
    stIdx_utc = getTimeIndex336(event.dtstart.value)
    enIdx_utc = getTimeIndex336(event.dtend.value)
    
    print 'Time (as a datetime object), local: ', stTime_local
    print 'startTimeIndex(UTC): ', stIdx_utc
    print 'Time (as a datetime object), local: ', enTime_local
    print 'endTimeIndex(UTC): ', enIdx_utc
    
    # Generate TaskLog
    tasklogs.append(createTaskLogFromGCal(userID, 200, event.dtstart.value))
    tasklogs.append(createTaskLogFromGCal(userID, 500, event.dtend.value))
    
    # Assume that one event doen not span over more than one week.
    # Make token for every timeslot, and weekday. 
    for token in tokens:
        visit_check = [0] * 7;
        
        for index_timeslot in range(stIdx_utc, enIdx_utc+1):
            if visit_check[index_timeslot/48] == 0:
                visit_check[index_timeslot/48] = 1;
                predtokens.append(createPredTokenFromGCal(userID, token, weekday=index_timeslot/48))
            predtokens.append(createPredTokenFromGCal(userID, token, daytime=index_timeslot%48))
    


In [ ]:
print 'Number of predTokens: ', len(predtokens)
for t in predtokens[0:5]:
    print t
    
print 'Number of TaskLogs: ', len(tasklogs)
for t in tasklogs[0:5]:
    print t

In [ ]:
token_collection = db.predicttokens;
token_collection.remove({'userId': userID})

for t in predtokens:    
    token_collection.insert_one(t)
    
num =0
for t in token_collection.find():
    print t
    num +=1
    if num == 10:
        break
        
tasklog_collection = db.tasklogs;
tasklog_collection.remove({'userId': userID})

for t in tasklogs:    
    tasklog_collection.insert_one(t)
    
num =0
for t in tasklog_collection.find():
    print t
    num +=1
    if num == 10:
        break
    



In [ ]:
range(0, 10, 2)

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

matplotlib.rc('font', family='AppleGothic')  # 한글 폰트 설정. 이렇게 해야 plot에 한글 안깨지고 잘 나옴.
# matplotlib.rc('font', family='NanumGothic')  # 한글 폰트 설정. 이렇게 해야 plot에 한글 안깨지고 잘 나옴.
# matplotlib.rcParams['font.family'] = 'NanumGothic'

def visualize(title, textinfo, score, timelabel):

    # Data wraparound. Append first few data at the end of the data.
    score = score + score[0:24]
    timelabel = timelabel + timelabel[0:24]
    
    
    # the histogram of the data
    plt.plot(range(len(score)), score)


    plt.xlabel('Timeslot')
    plt.xticks(range(len(timelabel)), timelabel, size='small', rotation=70)

    plt.ylabel('tfidf score')
    plt.title(title + textinfo)
    ymax = 1
    plt.axis([0, len(timelabel), 0, ymax])
    plt.grid(True)
    weekday_name = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
    for val_x in range(0, 48*7+1, 48):
        plt.annotate(weekday_name[(val_x/48)%7], xy=(val_x+20, ymax/2), xytext=(val_x+20, ymax/2), color='r', fontsize=20)
        plt.axvline(x=val_x, ymin=0, ymax = ymax, linewidth=2, color='r')
    
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(60, 5)
    
    plt.show()
    
    return fig

In [ ]:
# TEST Script
%matplotlib inline

from pymongo import MongoClient
import batchTimePrefScore as calc
reload(calc)
import tokenizer

import sys
reload(sys) # Encoding trick. Suppress encoding error when we pass Korean as sys.argv
sys.setdefaultencoding("utf-8")


# db_addr='128.199.166.149'
db_addr='localhost'
db_port=27340

client = MongoClient(db_addr, db_port)
# print client.database_names()
db = client.test
# print db.collection_names()

collection = db.predicttokens;

timelabel=[]
weekday_name = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']
for i in range(48*7):
    timelabel.append('%s %02d:%02d'%(weekday_name[i/48],(i%48)/2, i%2*30))

total_num = len(cal.vevent_list);
num = 0;
for event in cal.vevent_list:
    num += 1
    if num % 10 == 0:
        print '%d / %d'%(num, total_num)
    
    content = event.summary.value + ' ' + event.description.value
    tokens = tokenizer.extractor(content)

    score = calc.getTimePrefScore(userID, collection, tokens)

    import json

    f = visualize(content,json.dumps(tokens, ensure_ascii=False).encode('utf-8'), score, timelabel)
    f.savefig('image_log/'+event.summary.value+'.png', dpi=100)
